In [12]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
with open('dataset_raw/openai.api.key.2', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [14]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [15]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [16]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [17]:
def gen_code3(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(value_list = value_list, question = question), code)

def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

def gen_code_table(llm, messages, question, table): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"table": table, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(table = table, question = question), code)

def exec_code_table(code, table):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({table})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0, top_p = 1.0, seed=11)
llm = ChatOpenAI(model="gpt-3.5-turbo", 
temperature=0,
    #max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    seed=123)

In [225]:
[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a list of annotated values: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
            "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "To calculate the difference, use absolute value. " +
            "To calculate proportion, do not calculate percentage. " +
            "Use all given year values if no year specified. " +
            "The code must be specific to the provided value list. " +
            "Do not generate explanation, nor example code, just the function. "                  
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]   

messages_table=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a table. You will receive the financial report as a table and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a table: {table}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
            "Generate a Python function 'run(table)' that can answer the question using the table! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+ 
            "To select a row match the first cell value in code! "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "To calculate the difference, use absolute value. " +
            "To calculate proportion, do not calculate percentage. " +
            "Use all given year values if no year specified. " +
            "The code must be specific to the provided table. " +
            "Do not generate explanation, nor example code, just the function."
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ] 

In [226]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

qid = 'ba6783f3-8207-419a-b407-3f688682caef'
qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'

#qid = 'bf7abd62-d9cd-48d2-8826-1457684019a3'
#qid = '4d259081-6da6-44bd-8830-e4de0031744c'
#qid = 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'

#qid = '7cd3aedf-1291-4fea-bc9d-a25c65727b7b'
#qid = '22e20f25-669a-46b9-8779-2768ba391955' # 547 year avreage change 
#qid = '65ec782c-691e-45df-b541-caecb85154ff'
#qid = 'a983501d-2eec-486d-9661-e520c7c8af5e'
#qid = 'a81f1322-e74f-4e3c-a6cf-4b8d25d01cf5'
#qid = '64c902c6-f426-4432-84b3-c10b3065716f'
#qid = '6100c476-160a-4f1e-bfc1-a16f4cc18b52'
#qid = 'ef274d2b-fbd6-4e9f-95f9-79b37827d91c'
#qid = '94916f79-4b6d-487f-8fa2-39f81a085760'
#qid = '72325ec6-41ad-4648-9798-b22a61122cb4'
#qid = '263d03ec-83d2-48df-8376-1a72167798f7'
#qid = '4f7c8e0a-6ae9-40ee-bd16-06db6f96eaf1'
#qid = '35d602ae-9131-4291-a30c-49a40f32bbe4'

#qid = 'bd2d81eb-46fc-4e62-908d-aebfccf46246' # felreertheto sort is bevesz
#qid = '01de2123-400f-4411-98d9-cd3f9f1e4136' # szovegben is volt fontos ertek
#qid = 'fec503e8-6f91-483e-856d-bb1278bd031f' # zarojel problema
#qid = '59f13f19-4494-4c54-8e9a-1b499d50021a'
#qid = 'f12806ca-aa4c-4fe6-84fc-10cefca76a9e'
#qid = 'f36e5912-f63c-4837-a9ed-ddbe73e5148b'
#qid = 'eb50d3aa-16da-4087-9ab2-10106ebd10e7'
#qid = '8b2d5db2-7a47-4269-9df2-f44385bc0883'

#tablazat javitas utan megjelentek
#qid = '010df393-a62e-408d-888a-045a4e435e6a'
#qid = '73693527-ed4b-4d07-941e-0e654095a43d'
#qid = '426e47ae-aff7-43e2-81be-0bb951b0def1'
#qid = 'b8afd327-b836-4276-8556-eac89746b09f'
#qid = '6ba4045d-4222-4da0-a126-94b267a707ec'
#qid = 'e9b75381-50b1-4717-a322-31ed1eef4259'
#qid = 'd06a5ade-d848-4325-a2a5-8f5ef427d246'
#qid = '3502f875-f816-4a00-986c-fef9b08c0f96'

#qid = '99f83ba7-2e57-44d8-ac0a-64ab3e9c7b47' # subheader -t nem vette figyelembe

#Error
#qid = '5c8c999e-354f-4693-9b2d-29e3c03cb2af' # vonast nullanak kell ertelmezni? -
#qid = 'b382a11b-749b-425a-a77d-20e943e00f77' # +
#qid = 'e18e3335-7e7a-4138-9f1b-813604e44c3c'
#qid = 'e02b2c21-3972-41cc-b391-5478da7df9d8'
qid = '359f9c4c-b405-40e5-88b5-86bfa0069b89'
_table, _q =  get_question(qid)
table = _table['table']
print("tid: ",_table['uid'])
[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_multitable(table)


[print(v) for v in values]
#response = chain.invoke({"value_list": values, "question":q})
#code =  response.replace('```python','').replace('```','')
p, code = gen_code3(llm, messages, q, values)
#p, code = gen_code_table(llm, messages_table, q, table)
print(code)
#print(len(p) + len(code))

(v, s) = exec_code(code, values)
#(v, s) = exec_code_table(code, table)

#prompt_template = ChatPromptTemplate.from_messages(messages)
#prompt = prompt_template.format(dataset=values, question=q)

#prompt = ChatPromptTemplate.from_messages(messages_direct)

#output_parser = StrOutputParser()

#chain = prompt | llm | output_parser
    
#response = chain.invoke({"dataset": values, "question":q})
#print(response)
#code =  response.replace('```python','').replace('```','')
#(v, s) = exec_code(code, values)


rng = 0.01
if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
    print("\033[92m Success: " + str(v)+'\033[0m')
else:    
    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  

tid:  4c2f88e1-b5d4-4565-a98a-04f56127e4af
['', '', 'Years Ended December 31,', '']
['', '2019', '2018', '2017']
['Balance at beginning of period', '$13,162', '$15,990', '$11,401']
['Additions based on tax positions taken during a prior period', '484', '94', '1,258']
['Additions based on tax positions taken during a prior period - acquisitions', '4,479', '757', '—']
['Additions based on tax positions taken during the current period', '—', '—', '4,433']
['Reductions based on tax positions taken during a prior period', '(4,295)', '(153)', '—']
['Reductions related to a lapse of applicable statute of limitations', '(821)', '(3,144)', '(1,102)']
['Reductions related to a settlement with taxing authorities', '—', '(382)', '—']
['Balance at end of period', '$13,009', '$13,162', '$15,990']
What was the percentage change in the Reductions related to a lapse of applicable statute of limitations between 2017 and 2018?
{'number_value': 13162.0, 'scale': '', 'category': 'Balance at beginning of pe

In [22]:
pd.DataFrame(table)

,0,1,2,3
0,Income statement expense,,,
1,,2019 €m,2018 €m,2017 €m
2,Defined contribution schemes,166,178,192
3,Defined benefit schemes,57,44,20
4,Total amount charged to income statement (note...,223,222,212


In [23]:
pd.DataFrame(values)

,number_value,scale,category,header1,header2
0,166.0,million,Defined contribution schemes,2019,Income statement expense
1,178.0,million,Defined contribution schemes,2018,Income statement expense
2,192.0,million,Defined contribution schemes,2017,Income statement expense
3,57.0,million,Defined benefit schemes,2019,Income statement expense
4,44.0,million,Defined benefit schemes,2018,Income statement expense
5,20.0,million,Defined benefit schemes,2017,Income statement expense
6,223.0,million,Total amount charged to income statement (note...,2019,Income statement expense
7,222.0,million,Total amount charged to income statement (note...,2018,Income statement expense
8,212.0,million,Total amount charged to income statement (note...,2017,Income statement expense


In [49]:
qids = ['010df393-a62e-408d-888a-045a4e435e6a',
'73693527-ed4b-4d07-941e-0e654095a43d',
'426e47ae-aff7-43e2-81be-0bb951b0def1',
'b8afd327-b836-4276-8556-eac89746b09f',
'6ba4045d-4222-4da0-a126-94b267a707ec',
'e9b75381-50b1-4717-a322-31ed1eef4259']
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
    print(_q['uid'])
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (v, s) = exec_code(code, values)
    rng = 0.01
    if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )   

010df393-a62e-408d-888a-045a4e435e6a
 failure: -14212.0 good answer:  14212 
73693527-ed4b-4d07-941e-0e654095a43d
 failure: 0.02 good answer:  0.9500000000000001 
426e47ae-aff7-43e2-81be-0bb951b0def1
 failure: 259.8 good answer:  112.5 
b8afd327-b836-4276-8556-eac89746b09f
[Error]'[' was never closed (<string>, line 10)
 failure: [Error]'[' was never closed (<string>, line 10) good answer:  -6841 
6ba4045d-4222-4da0-a126-94b267a707ec
[Error]
 failure: [Error] good answer:  21.33 
e9b75381-50b1-4717-a322-31ed1eef4259
 failure: -1.0 good answer:  -2 


In [25]:
def get_answer(llm, messages, table, q):
    values = table_convert.convert_multitable(table)    
    p, code = gen_code3(llm, messages, q, values)    
    #print(code)
    (v, s) = exec_code(code, values)
    return  (v, s, code)

In [227]:
from tqdm.notebook import tqdm as log_progress

devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
cnt = 0
res = []
for i, item in log_progress(devdf.iterrows()):
    try:
        #if cnt == 36:
        #    break;
        #print(item['table']['uid'])
        table = item['table']['table']
        value_list = table_convert.convert_multitable(table)    
        
        for q in item['questions']:        
            if q['answer_type'] == 'arithmetic' and 'table' == q['answer_from']:   
                cnt = cnt + 1
                print(q['uid'])
                _table, _q =  get_question(q['uid'])
                
                table = _table['table']
                
                #[print(r) for r in table]
                
                q = _q['question']
                print(q, end='')
                
                
                r = []
                #for i in [0,1,2]:        
                #   r.append(get_answer(llm, messages, table, q))
                
                #if r[0][0] == r[1][0]:
                #    (v, s) = r[0]
                #elif r[1][0] == r[2][0]:
                #    (v, s) = r[1]
                #elif r[0][0] == r[2][0]:
                #    (v, s) = r[0]
                #else:
                #    (v, s) = (None, None)
                    
                (v, s, code) = get_answer(llm, messages, table, q)    
                
                rng = 0.01
                if  not isinstance(v, tuple) and not isinstance(v, str)  and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
                  print("\033[92m Success: " + str(v)+'\033[0m')
                else:    
                    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 
                
                
                (pred_value, pred_scale) = (v,s)
                if isinstance(pred_value, tuple) and len(pred_value) == 2:
                    print('$$$$')
                    (pred_value, pred_scale) = pred_value
                if pred_scale == "%" or pred_scale == "percentage"  :
                    pred_scale = 'percent'
                if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                    print('Invalid ', pred_scale)
                    pred_scale = ""    
                #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                #    pred_scale = res_scale[item['table']['uid']]
                err=""
                if isinstance(pred_value, str):
                    #print("string")
                    if  pred_value.startswith('[Error]'):
                        (pred_value, pred_scale) = ("", "")
                        err = v                                                        
                res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, pred_value, pred_scale, _q, code, table, value_list, err))
    except Exception as e:
        s = '[Outer Exception]'+ str(e)
        print(s)            

0it [00:00, ?it/s]

b2786c1a-37de-4120-b03c-32bf5c81f157
What was the change in the amount for Appliances in 2019 from 2018? failure: 94.0 good answer:  -94 
fe11f001-3bfe-4089-8108-412676f0a780
What was the percentage change in the amount for Appliances in 2019 from 2018? failure: 12.14 good answer:  -12.14 
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf
What is the difference between the domestic and international discount rates as at September 30, 2019? Success: 2.1
4dc8be43-d8d9-4b08-9ffd-9c19012361ce
What is the year on year percentage change in domestic discount rate between 2018 and 2019?

KeyboardInterrupt: 

In [26]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.5967741935483871 0.5967741935483871 0.6975806451612904


In [67]:
good = 0
table_only = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if q['answer_from'] == 'table':
        table_only = table_only + 1
    else:
        continue
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print(q['uid'], '|', q['question'],'|', q['derivation'], '|',err, '||', ans['answer'], '|', ans['scale'])
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] != pred_scale:
        #    print(q['question'])
        #    print(print ('SS',q['uid'],q['derivation'], pred, '[', pred_scale, '] <S> ' , ans['answer'], ans['scale']))
            
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'],'|', q['question'],'|', q['derivation'], '|',pred, '|',pred_scale, '|', ans['answer'], '|', ans['scale'])
        #print(q['question'])
        #print(value_list)
        #print(value_list)
        #print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
#print (good/len(res))
print (good/table_only)
print(good)

64c902c6-f426-4432-84b3-c10b3065716f | What was the difference in net profit between both FYs? | -9,819 - 6,639  | 3180.0 |  | -16458 | thousand
010df393-a62e-408d-888a-045a4e435e6a | What is the difference in amount between Deferred Revenue and Other non-current liabilities as reported? | (38,476-24,264) | -14212.0 |  | 14212 | thousand
5c8c999e-354f-4693-9b2d-29e3c03cb2af | What was the change in the preferred stock disposition from 2018 to 2019? | -9.9 - 0 | [Error] || -9.9 | percent
73693527-ed4b-4d07-941e-0e654095a43d | What was the employee termination costs as a proportion of total costs in 2018? | 53.0/55.5 | 0.02 |  | 0.9500000000000001 | 
ef9d4839-5277-4614-bb73-f902fd8a38b6 | What was the average Tax exempt interest income? | - (197 + 101 + 206) / 3 | 168.0 | thousand | -168 | thousand
fec503e8-6f91-483e-856d-bb1278bd031f | What was the percentage change in the Domestic manufacturers deduction from 2017 to 2018? | -2,545 / 4,095 - 1 | -37.85 | percent | -162.15 | percent
59f

In [29]:
len(res), table_only

(496, 496)

In [72]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    print(q['uid'], '{',pred_scale, '}', ans['scale'], ans['answer'], err, end='')
    if pred_scale == ans['scale']:
        print()
    else:
        print('<------------ XXX')
    

eb787966-fa02-401f-bfaf-ccabf3828b23 {  } million -12.6 <------------ XXX
05b670d3-5b19-438c-873f-9bf6de29c69e { percent } percent -22.22 
b2786c1a-37de-4120-b03c-32bf5c81f157 { million } million -94 
fe11f001-3bfe-4089-8108-412676f0a780 { percent } percent -12.14 
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf { percent } percent 2.1 
4dc8be43-d8d9-4b08-9ffd-9c19012361ce { percent } percent 6.67 
6c44a1a8-0785-43a0-90ab-7e21df2c57d9 { percent } percent -8.11 
a0414f81-8dc2-44b2-a441-2c9d9c805c4d { million } million 172 
bf7abd62-d9cd-48d2-8826-1457684019a3 { million } million 50.5 
4d259081-6da6-44bd-8830-e4de0031744c { million } million 121.5 
bed1fce2-69cb-4d1e-a34a-01950a1770bd { percent } percent 26.82 
348d031d-73ab-4b35-af46-998cfef25775 { percent } percent 25.1 
aea850af-68cb-4cc1-80b2-c9c2860b273e { percent } percent 2.98 
ba6783f3-8207-419a-b407-3f688682caef {  }  2.93 
263d03ec-83d2-48df-8376-1a72167798f7 {  }  0.87 
78fc6d55-c20c-4f71-99fe-bc40a16e61d0 {  }  -361 
35d602ae-9131-4291-

In [21]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if (isinstance(pred, float) and pred/1000000 ==  ans['answer'])  and pred != 0 :
        print(pred,ans['answer'])

In [28]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if pred ==  ans['answer']*-1 and pred != 0:
        print(pred,ans['answer'])
        #if ans['scale'] != pred_scale:

16458.0 -16458
-14212.0 14212
2.0 -2
-1226114.0 1226114
-29.84 29.84
-103.67 103.67
-112.0 112
6850.0 -6850
-1.9 1.9
-153391.0 153391
32074.0 -32074
6.1 -6.1
52.63 -52.63
-185.3 185.3
-11888.0 11888
31.58 -31.58
-236.0 236
-1503.0 1503
-15.5 15.5
-368.13 368.13


In [38]:
str(table)

"[['Years Ended December 31,', '', ''], ['', '2019', '2018'], ['Telecommunication - Maintenance', '$86.8', '$87.0'], ['Telecommunication - Installation', '33.2', '41.5'], ['Power - Operations, Maintenance & Construction Support', '19.9', '31.0'], ['Power - Cable Installation & Repair', '32.6', '34.8'], ['Total revenue from contracts with customers', '172.5', '194.3'], ['Other revenue', '—', '—'], ['Total Marine Services segment revenue', '$172.5', '$194.3']]"

In [84]:
good = 0
import re

for ans, pred, pred_scale, q,code, table,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    regex = re.compile("\\(\\d+,?\d+", re.VERBOSE)    
    match = regex.search(str(table))
    
    print (q['uid'],'|', q['question'],'|', q['derivation'], '|',pred, '|',pred_scale, '|', ans['answer'], '|', ans['scale'], '|',  'abs(' in code, '|',ans['answer'] == -1*pred, '|', isinstance(pred, float) and ( pred == _q['answer'] or (pred > llimit and pred < ulimit)), '|', match)
    #if pred ==  ans['answer']*-1 and pred != 0:
    #    print(pred,ans['answer'])
        #if ans['scale'] != pred_scale:

b2786c1a-37de-4120-b03c-32bf5c81f157 | What was the change in the amount for Appliances in 2019 from 2018? | 680-774 | -94.0 | million | -94 | million | False | False | False | None
fe11f001-3bfe-4089-8108-412676f0a780 | What was the percentage change in the amount for Appliances in 2019 from 2018? | (680-774)/774 | -12.14 | percent | -12.14 | percent | False | False | False | None
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf | What is the difference between the domestic and international discount rates as at September 30, 2019? | 4.00 - 1.90  | 2.1 | percent | 2.1 | percent | True | False | True | None
4dc8be43-d8d9-4b08-9ffd-9c19012361ce | What is the year on year percentage change in domestic discount rate between 2018 and 2019? | (4.00 - 3.75)/3.75  | 6.67 | percent | 6.67 | percent | False | False | True | None
6c44a1a8-0785-43a0-90ab-7e21df2c57d9 | What is the year on year percentage change in international expected return on plan assets between 2018 and 2019? | (3.40 - 3.70)/3.70  | -8.

In [94]:
res2 = []
for ans, pred, pred_scale, q,code, table,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    good = isinstance(pred, float) and ((pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit)      
    regex = re.compile("\\(\\d+,?\\d+", re.VERBOSE)    
    match = regex.search(str(table))
    match = match != None
    res2.append( {'qid': q['uid'], 'question' : q['question'], 'derivation': q['derivation'], 'pred' : pred, 'pred_scale': pred_scale,  'answer': ans['answer'],  'scale':  ans['scale'],  'abs': 'abs(' in code, 'inv' : ans['answer'] == -1*pred,  'good': good, 'parenth': match})
res2 = pd.DataFrame(res2)
res2[1:10]

,qid,question,derivation,pred,pred_scale,answer,scale,abs,inv,good,parenth
1,fe11f001-3bfe-4089-8108-412676f0a780,What was the percentage change in the amount f...,(680-774)/774,-12.14,percent,-12.14,percent,False,False,True,False
2,5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf,What is the difference between the domestic an...,4.00 - 1.90,2.1,percent,2.10,percent,True,False,True,False
3,4dc8be43-d8d9-4b08-9ffd-9c19012361ce,What is the year on year percentage change in ...,(4.00 - 3.75)/3.75,6.67,percent,6.67,percent,False,False,True,False
4,6c44a1a8-0785-43a0-90ab-7e21df2c57d9,What is the year on year percentage change in ...,(3.40 - 3.70)/3.70,-8.11,percent,-8.11,percent,False,False,True,False
5,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...,(166+178)/2,172.0,million,172.00,million,False,False,True,False
6,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?,(57+44)/2,50.5,million,50.50,million,False,False,True,False
7,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...,[(166+178)/2] - [(57+44)/2],121.5,million,121.50,million,True,False,True,False
8,bed1fce2-69cb-4d1e-a34a-01950a1770bd,What is the percentage change in the operating...,"(73,260-57,768)/57,768",26.82,percent,26.82,percent,True,False,True,True
9,aea850af-68cb-4cc1-80b2-c9c2860b273e,What is the percentage change in the net defer...,(795-772)/772,2.98,percent,2.98,percent,True,False,True,True


In [91]:
print('1')
print("full: ", len(res2), '\ngood: ', len(res2.query('good == True')),  '\nABS:', len(res2.query('abs == True')), '\nABS Good:', len(res2.query('abs == True and good == True')),
      '\nABS Bad:', len(res2.query('abs == True and good == False')), '\nABS Bad Inv:', len(res2.query('abs == True and inv == True')), 
      '\nPAR good: ', len(res2.query('good == True and parenth ==True')),    '\nPAR bad: ', len(res2.query('good == False and parenth ==True')), 
      '\nPAR ABS:', len(res2.query('abs == True  and parenth ==True'  )), '\nABS Good:', len(res2.query('abs == True and good == True and parenth ==True')),
      '\nPAR ABS Bad:', len(res2.query('abs == True and good == False  and parenth ==True')), '\nPAR ABS Bad Inv:', len(res2.query('abs == True and inv == True and parenth ==True'))
     )

1
full:  496 
good:  397 
ABS: 65 
ABS Good: 52 
ABS Bad: 13 
ABS Bad Inv: 6 
PAR good:  178 
PAR bad:  72 
PAR ABS: 43 
ABS Good: 31 
PAR ABS Bad: 12 
PAR ABS Bad Inv: 2


In [96]:
print('-1')
print("full: ", len(res2), '\ngood: ', len(res2.query('good == True')),  '\nABS:', len(res2.query('abs == True')), '\nABS Good:', len(res2.query('abs == True and good == True')),
      '\nABS Bad:', len(res2.query('abs == True and good == False')), '\nABS Bad Inv:', len(res2.query('abs == True and inv == True')), 
      '\nPAR good: ', len(res2.query('good == True and parenth ==True')),    '\nPAR bad: ', len(res2.query('good == False and parenth ==True')), 
      '\nPAR ABS:', len(res2.query('abs == True  and parenth ==True'  )), '\nPAR ABS Good:', len(res2.query('abs == True and good == True and parenth ==True')),
      '\nPAR ABS Bad:', len(res2.query('abs == True and good == False  and parenth ==True')), '\nPAR ABS Bad Inv:', len(res2.query('abs == True and inv == True and parenth ==True'))
     )

-1
full:  496 
good:  403 
ABS: 83 
ABS Good: 65 
ABS Bad: 18 
ABS Bad Inv: 12 
PAR good:  185 
PAR bad:  65 
PAR ABS: 60 
PAR ABS Good: 44 
PAR ABS Bad: 16 
PAR ABS Bad Inv: 7


In [97]:
res2.query('abs == True and good == False  and parenth ==True')

,qid,question,derivation,pred,pred_scale,answer,scale,abs,inv,good,parenth
17,a983501d-2eec-486d-9661-e520c7c8af5e,What was the average difference between EBITDA...,"((85,123 - 79,046) + (63,954 - 62,575)) / 2",2349.0,,3728.00,thousand,True,False,False,True
18,64c902c6-f426-4432-84b3-c10b3065716f,What was the difference in net profit between ...,"-9,819 - 6,639",16458.0,,-16458.00,thousand,True,True,False,True
41,94ef7822-a201-493e-b557-a640f4ea4d83,What is the change in the average total curren...,"[(1,062+1,018)/2] - [(1,018+950)/2]",34.0,million,56.00,million,True,False,False,True
64,147f7c09-a627-4719-93ad-33661b5a4543,"What was the change in Total interest expense,...",138-140,2.0,million,-2.00,million,True,True,False,True
115,732c81f8-a16d-4d34-9917-fa98c195feec,What was the percentage change in accounts pay...,"[(-18,668) - (-9,166)] / -9,166",-103.67,percent,103.67,percent,True,True,False,True
199,8c882f0d-684c-430e-8194-997c565a7258,What is the nominal difference for basic earni...,206.2 - 132.6,969.0,,73.60,,True,False,False,True
237,524463b6-b8d7-410d-926a-699f7b7cdd99,What is the difference between the net cash us...,"(-42,056)-(-9,982)",32074.0,,-32074.00,million,True,True,False,True
272,e4f21617-1a84-40e4-bdf0-671df88cb862,What is the percentage change in the working c...,(18-38)/38,52.63,percent,-52.63,percent,True,True,False,True
277,359f9c4c-b405-40e5-88b5-86bfa0069b89,What was the percentage change in the Reductio...,"(-3,144-(-1,102))/-1,102",-185.3,percent,185.30,percent,True,True,False,True
325,5f289981-f13d-4160-8879-c81dc0b7381e,What was the percentage change in the Share ba...,"(4,556-4,296)/4,296",,,6.05,percent,True,False,False,True


In [220]:
messages=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "You will receive the financial report as a dataset and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "
         ),
        (
          "human",
          "Here is the financial report as a dataset: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the dataset."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
            "Generate a Python function 'run(dataset)' that can answer the question using the dataset! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            #"Follow these rules while generating the code: "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            #"If the question is about calculating the percentage change, use equation: percentage_change = (signed_new_value - signed_old_value)/signed_old_value * 100. " +
            "To calculate percentage change, use equation: percentage_change = (2*signed_new_value - 2*signed_old_value)/2*signed_old_value * 100. " +
            "To calculate the difference, use absolute value." +                        
            #"To calculate absolute change, use abs function." +                        
            #"To calculate percentage change, remove the abs function from calculation." +            
            "To calculate proportion, do not calculate percentage. " +
            #"- Use all given year values if no year specified. " +
            "The code must be specific to the provided dataset. " +
            "Do not generate explanation, nor example code, just the function. "                  
            ""
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]  

In [222]:
qid = '732c81f8-a16d-4d34-9917-fa98c195feec'
qid = 'e4f21617-1a84-40e4-bdf0-671df88cb862'
_table, _q =  get_question(qid)
table = _table['table']
print("tid: ",_table['uid'])
[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_multitable(table)

#[print(v) for v in values]
p, code = gen_code3(llm, messages, q, values)
print(code)

(v, s) = exec_code(code, values)


rng = 0.01
if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
    print("\033[92m Success: " + str(v)+'\033[0m')
else:    
    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  

tid:  0027cf6e-f6e8-4d8b-b4ee-0b9f9aeb1f54
['', 'Estimated at June 30, 2019', 'Adjustments', 'Final as of December 31, 2019']
['Cash', '$3,795', '$ -', '$3,795']
['Working capital adjustment to purchase price', '(38)', '20', '(18)']
['Total fair value of consideration transferred', '3,757', '20', '3,777']
['Accounts receivable', '591', '-', '591']
['Inventories', '149', '-', '149']
['Deposits and other current assets', '4', '8', '12']
['Property and equipment', '1,560', '-', '1,560']
['Customer relationship', '930', '-', '930']
['Other finite-lived intangible assets', '35', '-', '35']
['Accounts payable', '(219)', '-', '(219)']
['Finance lease liabilities', '(18)', '-', '(18)']
['Net recognized amounts of identifiable assets acquired and liabilities assumed', '3,032', '8', '3,040']
['Goodwill', '$                          725', '$                            12', '$                          737']
What is the percentage change in the working capital adjustment at June 30 and December 31,

In [223]:
llm('2019 income was -18668 USD, 2018 income was -9166 USD. What was the percentage change in income?')

AIMessage(content='To calculate the percentage change in income, we first need to find the difference between the two incomes:\n\nDifference = 2019 income - 2018 income\nDifference = -18668 USD - (-9166 USD)\nDifference = -18668 USD + 9166 USD\nDifference = -9502 USD\n\nNext, we calculate the percentage change using the formula:\n\nPercentage Change = (Difference / |2018 income|) * 100\n\nPercentage Change = (-9502 USD / 9166 USD) * 100\nPercentage Change = -1.037 * 100\nPercentage Change = -103.7%\n\nTherefore, the percentage change in income from 2018 to 2019 was a decrease of 103.7%.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 34, 'total_tokens': 185, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 

In [224]:
llm('2019 income was 18 USD, 2018 income was 38 USD. What was the percentage change in income?')

AIMessage(content='To calculate the percentage change in income, you can use the following formula:\n\nPercentage Change = ((New Value - Old Value) / Old Value) * 100\n\nIn this case:\nNew Value = 18 USD\nOld Value = 38 USD\n\nPercentage Change = ((18 - 38) / 38) * 100\nPercentage Change = (-20 / 38) * 100\nPercentage Change = -0.5263 * 100\nPercentage Change = -52.63%\n\nTherefore, the percentage change in income from 2018 to 2019 was a decrease of 52.63%.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 32, 'total_tokens': 158, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7e480e9c-a0ee-44b7-805e-51f23c583e0a-0', usage_metadata={'input_tokens': 3

In [176]:
llm('2019 income was -18668 USD, 2018 income was -9166 USD. What was the percentage change in income? Use equation: percentage_change = ((signed_new_value - signed_old_value)/signed_old_value) * 100. )

AIMessage(content='To calculate the percentage change in income, we first need to find the difference between the two incomes:\n\nDifference = 2019 income - 2018 income\nDifference = -18668 USD - (-9166 USD)\nDifference = -18668 USD + 9166 USD\nDifference = -9502 USD\n\nNext, we calculate the percentage change using the formula:\n\nPercentage Change = (Difference / |2018 income|) * 100\nPercentage Change = (-9502 USD / 9166 USD) * 100\nPercentage Change = -1.037 * 100\nPercentage Change = -103.7%\n\nTherefore, the percentage change in income from 2018 to 2019 was a decrease of 103.7%.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 35, 'total_tokens': 186, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': No

In [174]:
ms = [
    ('human', "2019 income was -18668 USD, 2018 income was -9166 USD. What was the percentage change in income?"),
    ('ai','To calculate the percentage change in income, we first need to find the difference between the two incomes:\n\nDifference = 2019 income - 2018 income\nDifference = -18668 USD - (-9166 USD)\nDifference = -18668 USD + 9166 USD\nDifference = -9502 USD\n\nNext, we calculate the percentage change using the formula:\n\nPercentage Change = (Difference / |2018 income|) * 100\nPercentage Change = (-9502 USD / 9166 USD) * 100\nPercentage Change = -1.037 * 100\nPercentage Change = -103.7%\n\nTherefore, the percentage change in income from 2018 to 2019 was a decrease of 103.7%.'),
    ('human','Why did you use absolute value in the denominator? ')
]

llm(ms)

AIMessage(content="I apologize for the mistake in my previous calculation. The absolute value should not be used in the denominator when calculating the percentage change in income. \n\nLet's recalculate the percentage change without using the absolute value:\n\nPercentage Change = (Difference / 2018 income) * 100\nPercentage Change = (-9502 USD / -9166 USD) * 100\nPercentage Change = 1.036 * 100\nPercentage Change = 103.6%\n\nTherefore, the correct percentage change in income from 2018 to 2019 is a decrease of 103.6%. Thank you for pointing out the error.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 203, 'total_tokens': 330, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop',

In [194]:
ms = [
    ('human', "2019 income was -18668 USD, 2018 income was -9166 USD. What was the percentage change in income?"),
    ('ai','To calculate the percentage change in income, we first need to find the difference between the two incomes:\n\nDifference = 2019 income - 2018 income\nDifference = -18668 USD - (-9166 USD)\nDifference = -18668 USD + 9166 USD\nDifference = -9502 USD\n\nNext, we calculate the percentage change using the formula:\n\nPercentage Change = (Difference / |2018 income|) * 100\nPercentage Change = (-9502 USD / 9166 USD) * 100\nPercentage Change = -1.037 * 100\nPercentage Change = -103.7%\n\nTherefore, the percentage change in income from 2018 to 2019 was a decrease of 103.7%.'),
    ('human','Why did not you use absolute value in the nominator? ')
]
llm(ms)

AIMessage(content="I apologize for the oversight. When calculating percentage change, it is important to use the absolute value of the difference in income in the numerator to ensure that the percentage change is always positive, regardless of whether the income increased or decreased.\n\nLet's recalculate the percentage change using the absolute value of the difference in income in the numerator:\n\nDifference = 2019 income - 2018 income\nDifference = -18668 USD - (-9166 USD)\nDifference = -18668 USD + 9166 USD\nDifference = -9502 USD\n\nPercentage Change = (|Difference| / |2018 income|) * 100\nPercentage Change = (|-9502 USD| / 9166 USD) * 100\nPercentage Change = (9502 USD / 9166 USD) * 100\nPercentage Change = 1.033 * 100\nPercentage Change = 103.3%\n\nTherefore, the correct percentage change in income from 2018 to 2019 is a decrease of 103.3%. Thank you for pointing out the error.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 216, '

In [211]:
qids = ['732c81f8-a16d-4d34-9917-fa98c195feec',
'8c882f0d-684c-430e-8194-997c565a7258',
'524463b6-b8d7-410d-926a-699f7b7cdd99',
'e4f21617-1a84-40e4-bdf0-671df88cb862',
'359f9c4c-b405-40e5-88b5-86bfa0069b89',
'5f289981-f13d-4160-8879-c81dc0b7381e',
'ed47e72c-c67c-4c61-abfa-9aefcf4caa89']
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
    print(_q['uid'])
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (v, s) = exec_code(code, values)
    rng = 0.01
    if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 



732c81f8-a16d-4d34-9917-fa98c195feec
 Success: 103.67
8c882f0d-684c-430e-8194-997c565a7258
 failure: -9.1 good answer:  73.6 
524463b6-b8d7-410d-926a-699f7b7cdd99
 failure: 32074.0 good answer:  -32074 
e4f21617-1a84-40e4-bdf0-671df88cb862
 failure: 52.63 good answer:  -52.63 
359f9c4c-b405-40e5-88b5-86bfa0069b89
 Success: 185.3
5f289981-f13d-4160-8879-c81dc0b7381e
[Error]division by zero
 failure: [Error]division by zero good answer:  6.05 
ed47e72c-c67c-4c61-abfa-9aefcf4caa89
 failure: 25.6 good answer:  -0.26 
